In [1]:
import requests
from PIL import Image
from io import BytesIO
import os
import pandas as pd
from datetime import datetime
import numpy as np
import time
import torch

In [ ]:
# Load Abandoned Mine database from Western Australian Government Department Energy, Mines Indstry....
# https://catalogue.data.wa.gov.au/dataset/abandoned-mines
# Downloaded September 2024
#


am_path = '/Sample Data/'
am_filename = 'wabmines_GDA2020.csv'
am_labels = pd.read_csv(am_path+am_filename)
# display(am_labels)
am_labels2 = am_labels[['SITE_CODE','SITE_TYPE','SITE_SUB_T','LATITUDE','LONGITUDE']].copy()
am_labels2['Stage'] = 'Abandoned'
am_labels2['Stage2'] = 'Abandoned'

am_labels2  = am_labels2.rename(columns =  {  'SITE_CODE':	'SiteCode',
                                            'SITE_TYPE': 'Type',
                                            'SITE_SUB_T': 'SubType',
                                            'LATITUDE': 'Latitude',
                                            'LONGITUDE':'Longitude'})
am_labels2 = am_labels2[['SiteCode','Type','SubType','Stage','Latitude','Longitude','Stage2']]
display(am_labels2)
print(am_labels2.columns)

C:\Users\afrob\AppData\Local\Temp\ipykernel_26228\2988610291.py:3: DtypeWarning: Columns (11,12,13,33) have mixed types. Specify dtype option on import or set low_memory=False.
  am_labels = pd.read_csv(am_path+am_filename)


In [ ]:

# Load Active mine site data from Mine and Mineral Deposite database from Western Australian Government Department Energy, Mines Indstry....
# https://catalogue.data.wa.gov.au/dataset/minedex-dmirs-001
# Downloaded September 2024

act_path = '/Sample Data/'
act_filename = 'Sites.csv'
act_labels = pd.read_csv(act_path+act_filename)
# display(act_labels)
act_labels = act_labels[['SiteCode','Type','SubType','Stage','Latitude','Longitude']]
act_labels['Stage2'] = 'Active'
display(act_labels)

,SiteCode,Type,SubType,Stage,Latitude,Longitude,Stage2
0,S0000001,Mine,Openpit,Care and Maintenance,-28.565078,122.480754,Active
1,S0000002,Deposit,Unspecified,Undeveloped,-26.787357,118.574415,Active
2,S0000002,Deposit,Unspecified,Undeveloped,-26.787357,118.574415,Active
3,S0000005,Mine,Openpit,Shut,-29.690617,121.012812,Active
4,S0000006,Mine,Openpit,Shut,-30.239138,121.222793,Active
...,...,...,...,...,...,...,...
47090,S0244127,Mine,Openpit,Shut,-30.380134,121.054678,Active
47091,S0244128,Infrastructure,Dump (waste),Shut,-30.378254,121.056179,Active
47092,S0244129,Infrastructure,Dump (waste),Shut,-30.380064,121.058075,Active
47093,S0244130,Prospect,Drillhole,Undeveloped,-28.675087,117.610910,Active


In [ ]:
# Join active and abandoned mine site data
MineLabels = pd.concat((am_labels2,act_labels))
MineLabels = MineLabels.reset_index(drop=True)
display(MineLabels)

,SiteCode,Type,SubType,Stage,Latitude,Longitude,Stage2
0,S0130958,Other,Unspecified,Abandoned,-26.629076,120.238380,Abandoned
1,S0130959,Other,Unspecified,Abandoned,-26.629406,120.238060,Abandoned
2,S0130960,Mine,Shaft,Abandoned,-27.428387,117.866530,Abandoned
3,S0130961,Other,Unspecified,Abandoned,-26.629566,120.238050,Abandoned
4,S0130962,Other,Unspecified,Abandoned,-26.629706,120.238250,Abandoned
...,...,...,...,...,...,...,...
239613,S0244127,Mine,Openpit,Shut,-30.380134,121.054678,Active
239614,S0244128,Infrastructure,Dump (waste),Shut,-30.378254,121.056179,Active
239615,S0244129,Infrastructure,Dump (waste),Shut,-30.380064,121.058075,Active
239616,S0244130,Prospect,Drillhole,Undeveloped,-28.675087,117.610910,Active


In [ ]:
# Create matrix containing coordinates for images of 500x500m  

#converts meters to Latitude-Longitude
ImageSize =500.0
ImageStep = float(ImageSize/111139)
print(ImageStep)
image_number = 0

# starting point, lower left
LatStart = -31.0
LongStart = 118.0

# end point, upper right
LatEnd = -21.0
LongEnd = 128.0

#Create image template

LatRowStart = np.arange(start=LatStart,
                   stop=(LatEnd-ImageStep),
                   step = ImageStep,
                   dtype=float)
LatRowEnd = np.arange(start=(LatStart+ImageStep),
                   stop=LatEnd,
                   step = ImageStep,
                   dtype=float)
LongColStart = np.arange(start=LongStart,
                   stop=(LongEnd-ImageStep),
                   step = ImageStep,
                   dtype=float)
LongColEnd = np.arange(start=(LongStart+ImageStep),
                   stop=LongEnd,
                   step = ImageStep,
                   dtype=float)
print("LatRS",LatRowStart.shape)
print("LatRE",LatRowEnd.shape)

print("LatRS",LatRowStart)
print("LatRE",LatRowEnd)

print("LongRS",LongColStart.shape)
print("LongRE",LongColEnd.shape)

print("LongRS",LongColStart)
print("LongEnd",LongColEnd)



LatRS (2222,)
LatRE (2222,)
LatRS [-31.         -30.99550113 -30.99100226 ... -21.01700573 -21.01250686
 -21.00800799]
LatRE [-30.99550113 -30.99100226 -30.98650339 ... -21.01250686 -21.00800799
 -21.00350912]
LongRS (2222,)
LongRE (2222,)
LongRS [118.         118.00449887 118.00899774 ... 127.98299427 127.98749314
 127.99199201]
LongEnd [118.00449887 118.00899774 118.01349661 ... 127.98749314 127.99199201
 127.99649088]


In [8]:
nrow = LongColStart.shape[0]
ncol = LatRowStart.shape[0]

# IT = np.zeros((nrow,ncol,4))
# print(IT.shape)


ax1 = np.tile(LatRowStart,ncol).reshape((nrow,ncol))
ax2 = np.tile(LongColStart,nrow).reshape((nrow,ncol)).T 
ax3 = np.tile(LatRowEnd,ncol).reshape((nrow,ncol))
ax4 = np.tile(LongColEnd,nrow).reshape(nrow,ncol).T
IT = np.array((ax1,ax2,ax3,ax4))
IT = IT.reshape((4,nrow*ncol))
IT  = np.moveaxis(IT,0,1)
status_col = np.zeros((IT.shape[0],1))           
IT = np.concatenate((IT,status_col,status_col),axis =1)
print(IT.shape)
print(IT[0:10,:])



(4937284, 6)
[[-31.         118.         -30.99550113 118.00449887   0.
    0.        ]
 [-30.99550113 118.         -30.99100226 118.00449887   0.
    0.        ]
 [-30.99100226 118.         -30.98650339 118.00449887   0.
    0.        ]
 [-30.98650339 118.         -30.98200452 118.00449887   0.
    0.        ]
 [-30.98200452 118.         -30.97750565 118.00449887   0.
    0.        ]
 [-30.97750565 118.         -30.97300678 118.00449887   0.
    0.        ]
 [-30.97300678 118.         -30.9685079  118.00449887   0.
    0.        ]
 [-30.9685079  118.         -30.96400903 118.00449887   0.
    0.        ]
 [-30.96400903 118.         -30.95951016 118.00449887   0.
    0.        ]
 [-30.95951016 118.         -30.95501129 118.00449887   0.
    0.        ]]


In [ ]:
#set template for dims of images with mines

pd.options.display.float_format = '{:.10f}'.format

# Mine database contains the coorinates for the center of the mine site.
# Calculate coordinates for 500x500, around the center location.
# Add Lower left and upper right coordinates

MineLabels["LatStart"] = MineLabels["Latitude"]-ImageStep/2
MineLabels["LongStart"] = MineLabels["Longitude"]-ImageStep/2
MineLabels["LatEnd"] = MineLabels["Latitude"]+ImageStep/2
MineLabels["LongEnd"] = MineLabels["Longitude"]+ImageStep/2
display(MineLabels)

# Extract unique label classes and add back to dataframe as an ID
classes = pd.DataFrame(pd.unique(MineLabels['SubType']),columns = ['SubType'])
classes['ID'] = classes.index
# print(classes)
# #save for external use
# classes.to_csv("MineTypes.csv",index=False)    
MineLabels = pd.merge(left=MineLabels,right = classes, how="left",on='SubType')
display(MineLabels)

# convert mind labels dataframe to numpy array
Mine_array = np.array((MineLabels.index,MineLabels.LatStart,MineLabels.LongStart,MineLabels.LatEnd,MineLabels.LongEnd,MineLabels.ID)).T
# print(Mine_array.shape)
# print(Mine_array[0:3,:])
# print(IT[0:3,:])

,SiteCode,Type,SubType,Stage,Latitude,Longitude,Stage2,LatStart,LongStart,LatEnd,LongEnd
0,S0130958,Other,Unspecified,Abandoned,-26.6290763545,120.2383799007,Abandoned,-26.6313257899,120.2361304653,-26.6268269191,120.2406293361
1,S0130959,Other,Unspecified,Abandoned,-26.6294063543,120.2380599010,Abandoned,-26.6316557897,120.2358104656,-26.6271569189,120.2403093364
2,S0130960,Mine,Shaft,Abandoned,-27.4283870384,117.8665300418,Abandoned,-27.4306364738,117.8642806064,-27.4261376030,117.8687794772
3,S0130961,Other,Unspecified,Abandoned,-26.6295663541,120.2380499012,Abandoned,-26.6318157895,120.2358004658,-26.6273169187,120.2402993366
4,S0130962,Other,Unspecified,Abandoned,-26.6297063540,120.2382499013,Abandoned,-26.6319557894,120.2360004659,-26.6274569186,120.2404993367
...,...,...,...,...,...,...,...,...,...,...,...
239613,S0244127,Mine,Openpit,Shut,-30.3801340106,121.0546777003,Active,-30.3823834460,121.0524282649,-30.3778845752,121.0569271357
239614,S0244128,Infrastructure,Dump (waste),Shut,-30.3782540043,121.0561786900,Active,-30.3805034397,121.0539292546,-30.3760045689,121.0584281254
239615,S0244129,Infrastructure,Dump (waste),Shut,-30.3800640022,121.0580746819,Active,-30.3823134376,121.0558252466,-30.3778145668,121.0603241173
239616,S0244130,Prospect,Drillhole,Undeveloped,-28.6750865658,117.6109102117,Active,-28.6773360012,117.6086607763,-28.6728371305,117.6131596471


                      SubType  ID
0                 Unspecified   0
1                       Shaft   1
2            Shallow workings   2
3                    Building   3
4                       Stope   4
5          Dump (unspecified)   5
6              Costean/Trench   6
7                Dump (waste)   7
8                         Dam   8
9                   Drillhole   9
10                      Other  10
11  Tailings storage facility  11
12                    Openpit  12
13           Processing Plant  13
14           Headframe/Winder  14
15             Heap leach pad  15
16          Town/Village/Camp  16
17                       Adit  17
18                    Decline  18
19                       Well  19
20    Scraping-Detecting area  20
21            Dryblowing area  21
22                       Road  22
23                       Port  23
24                Environment  24
25                    Railway  25
26   Handling-loading-storage  26
27                    Outcrop  27
28            

,SiteCode,Type,SubType,Stage,Latitude,Longitude,Stage2,LatStart,LongStart,LatEnd,LongEnd,ID
0,S0130958,Other,Unspecified,Abandoned,-26.6290763545,120.2383799007,Abandoned,-26.6313257899,120.2361304653,-26.6268269191,120.2406293361,0
1,S0130959,Other,Unspecified,Abandoned,-26.6294063543,120.2380599010,Abandoned,-26.6316557897,120.2358104656,-26.6271569189,120.2403093364,0
2,S0130960,Mine,Shaft,Abandoned,-27.4283870384,117.8665300418,Abandoned,-27.4306364738,117.8642806064,-27.4261376030,117.8687794772,1
3,S0130961,Other,Unspecified,Abandoned,-26.6295663541,120.2380499012,Abandoned,-26.6318157895,120.2358004658,-26.6273169187,120.2402993366,0
4,S0130962,Other,Unspecified,Abandoned,-26.6297063540,120.2382499013,Abandoned,-26.6319557894,120.2360004659,-26.6274569186,120.2404993367,0
...,...,...,...,...,...,...,...,...,...,...,...,...
239613,S0244127,Mine,Openpit,Shut,-30.3801340106,121.0546777003,Active,-30.3823834460,121.0524282649,-30.3778845752,121.0569271357,12
239614,S0244128,Infrastructure,Dump (waste),Shut,-30.3782540043,121.0561786900,Active,-30.3805034397,121.0539292546,-30.3760045689,121.0584281254,7
239615,S0244129,Infrastructure,Dump (waste),Shut,-30.3800640022,121.0580746819,Active,-30.3823134376,121.0558252466,-30.3778145668,121.0603241173,7
239616,S0244130,Prospect,Drillhole,Undeveloped,-28.6750865658,117.6109102117,Active,-28.6773360012,117.6086607763,-28.6728371305,117.6131596471,9


(239618, 6)
[[  0.         -26.63132579 120.23613047 -26.62682692 120.24062934
    0.        ]
 [  1.         -26.63165579 120.23581047 -26.62715692 120.24030934
    0.        ]
 [  2.         -27.43063647 117.86428061 -27.4261376  117.86877948
    1.        ]]
[[-31.         118.         -30.99550113 118.00449887   0.
    0.        ]
 [-30.99550113 118.         -30.99100226 118.00449887   0.
    0.        ]
 [-30.99100226 118.         -30.98650339 118.00449887   0.
    0.        ]]


In [ ]:
#used for random sampling experiments.

# num_samples = 20000

# #random sample x amount
# sample_numbers = np.random.randint(0,IT.shape[0],num_samples)
# IT_sample = IT[sample_numbers,:]

# #first x samples
# # IT_sample = IT[0:num_samples,:]
# print(IT_sample.shape)
# print(IT_sample[0:3,0:5])

In [ ]:
# Toy test data

# IT_sample = [[-10,1,-8,3,1],
#              [-8,2,-6,4,1],
#              [-6,3,-4,5,1],
#              [-6,1,-4,3,1],
#              [-4,7,-2,9,1]]
# Mine_array = [[1,-7,2,-5,4],
#               [1,-7,3,-5,5]]

In [ ]:
# More realistic test data

# Mine_array = [[1,-26.62232804833,120.24512820687,-26.61782917753,120.24962707767],
#              [1,-26.64032353147,120.22713272373,-26.63582466067,120.23163159453],
#              [1,-26.63357522529,120.23388102991,-26.62907635449,120.23837990071],
#              [1,-26.62907635451,120.23837990069,-26.62457748371,120.24287877149],
#              [1,-26.62682691910,120.24062933610,-26.62232804832,120.24512820688]]
# IT_sample = [[-26.63132578990,120.23613046530,-26.62682691910,120.24062933610,1],
#              [1,-27.43063647380,117.86428060640,-27.42613760300,117.86877947720]]

In [ ]:
# To determine which base images have no mines, need to compare where mine sites overlap with base images.
# Compares coordinates of mine sites (500x500 around the center of site) to base images. 
# Flags images where images overlap and records the site IDs and types of sites for the
# overalapping images
#
# Uses a vectoized approach. Comparing 4.9 million x 295K x 8 conditions =  computationally heavy
# Iterated through base images in batches of 100. This was the faster approach on a NVIDIA GeForce RTX 3050  8mb GPU.
# 
# 


torch.set_printoptions(precision=10)

num_samples = IT.shape[0] #20000
iter_size = 100

# # randomly sample for smaller datasets
# sample_numbers = np.random.randint(0,IT.shape[0],num_samples)
# IT_sample = IT[sample_numbers,:]


for i in range(0,num_samples,iter_size):

    print("Iteration:",i,"to",i+iter_size)

    # convert to tensors to run in GPU
    IT_t = torch.tensor(IT[i:(i+iter_size),:]).to(device="cuda")
    MA_t = torch.tensor(Mine_array).to(device="cuda")

    num_images = IT_t.shape[0]
    # print(num_images)
    num_mines = MA_t.shape[0]
    # print(num_mines)


    # Mine Image Lower Left -> Upper-Lower edges of base images
    # Broadcasts Mine and base Images to same dims
    # Compares if >= (ge), <= (le), > (gt) & < (lt)
    # returns boolean tensor

    Res_LL1= torch.ge(torch.broadcast_to(MA_t[:,1][:,None],(num_mines,num_images)),
                    torch.broadcast_to(IT_t[:,0],(num_mines,num_images)))
    
    Res_LL2= torch.lt(torch.broadcast_to(MA_t[:,1][:,None],(num_mines,num_images)),
                    torch.broadcast_to(IT_t[:,2],(num_mines,num_images)))
   
    # Mine Image Lower Left -> Left-Right edges of base images
    Res_LL3= torch.ge(torch.broadcast_to(MA_t[:,2][:,None],(num_mines,num_images)),
                    torch.broadcast_to(IT_t[:,1],(num_mines,num_images)))
  
    Res_LL4= torch.lt(torch.broadcast_to(MA_t[:,2][:,None],(num_mines,num_images)),
                    torch.broadcast_to(IT_t[:,3],(num_mines,num_images)))
   
    # Upper Left -> Upper-Lower
    Res_UL1= torch.gt(torch.broadcast_to(MA_t[:,3][:,None],(num_mines,num_images)),
                    torch.broadcast_to(IT_t[:,0],(num_mines,num_images)))
  
    Res_UL2= torch.le(torch.broadcast_to(MA_t[:,3][:,None],(num_mines,num_images)),
                    torch.broadcast_to(IT_t[:,2],(num_mines,num_images)))
    
    # Upper Left -> Left-Right
    Res_UL3= torch.ge(torch.broadcast_to(MA_t[:,2][:,None],(num_mines,num_images)),
                    torch.broadcast_to(IT_t[:,1],(num_mines,num_images)))
 
    Res_UL4= torch.lt(torch.broadcast_to(MA_t[:,2][:,None],(num_mines,num_images)),
                    torch.broadcast_to(IT_t[:,3],(num_mines,num_images)))
    
    # Upper Right -> Upper-Lower
    Res_UR1 = Res_UL1
            # duplicate calc :    
            # torch.all(torch.gt(torch.broadcast_to(MA_t[:,3][:,None],(num_mines,num_images)),
                                # torch.broadcast_to(IT_t[:,0],(num_mines,num_images))),axis=0)
  
    Res_UR2 = Res_UL2
        # duplicate calc :
        # torch.all(torch.le(torch.broadcast_to(MA_t[:,3][:,None],(num_mines,num_images)),
                            # torch.broadcast_to(IT_t[:,2],(num_mines,num_images))),axis=0)
 
    # Upper Right -> Left-Right
    Res_UR3= torch.gt(torch.broadcast_to(MA_t[:,4][:,None],(num_mines,num_images)),
                                torch.broadcast_to(IT_t[:,1],(num_mines,num_images)))
 
    Res_UR4= torch.le(torch.broadcast_to(MA_t[:,4][:,None],(num_mines,num_images)),
                                torch.broadcast_to(IT_t[:,3],(num_mines,num_images)))
 
    # Lower Right -> Upper-Lower
    Res_LR1= Res_LL1
        # duplicate calc:
        #torch.all(torch.ge(torch.broadcast_to(MA_t[:,1][:,None],(num_mines,num_images)),
        #                       torch.broadcast_to(IT_t[:,0],(num_mines,num_images))),axis=0)
 
    Res_LR2= Res_LL2
        # duplicate calc :
        # torch.all(torch.lt(torch.broadcast_to(MA_t[:,1][:,None],(num_mines,num_images)),
        #                             torch.broadcast_to(IT_t[:,2],(num_mines,num_images))),axis=0)

    # Lower Right -> Left-Right
    Res_LR3= Res_UR3
        # duplicate calc :
        # torch.all(torch.gt(torch.broadcast_to(MA_t[:,4][:,None],(num_mines,num_images)),
        #                             torch.broadcast_to(IT_t[:,1],(num_mines,num_images))),axis=0)
    
    Res_LR4= Res_UR4
        # duplicate calc :
        # torch.all(torch.le(torch.broadcast_to(MA_t[:,4][:,None],(num_mines,num_images)),
                                    # torch.broadcast_to(IT_t[:,3],(num_mines,num_images))),axis=0)
   
   #checks if an overlap between the mine image dimensions and upper-lower and left-right cases
   # for each corner of the mine image, must be all inside to be true (torch.all)
    Res_LL = torch.all(torch.cat((Res_LL1.T.reshape(num_images*num_mines,1),Res_LL2.T.reshape(num_images*num_mines,1),Res_LL3.T.reshape(num_images*num_mines,1),Res_LL4.T.reshape(num_images*num_mines,1)),dim=1),dim=1)
    Res_UL = torch.all(torch.cat((Res_UL1.T.reshape(num_images*num_mines,1),Res_UL2.T.reshape(num_images*num_mines,1),Res_UL3.T.reshape(num_images*num_mines,1),Res_UL4.T.reshape(num_images*num_mines,1)),dim=1),dim=1) #.reshape((num_mines,4,num_images)),axis=1)
    Res_UR = torch.all(torch.cat((Res_UR1.T.reshape(num_images*num_mines,1),Res_UR2.T.reshape(num_images*num_mines,1),Res_UR3.T.reshape(num_images*num_mines,1),Res_UR4.T.reshape(num_images*num_mines,1)),dim=1),dim=1) #.reshape((num_mines,4,num_images)),axis=1)
    Res_LR = torch.all(torch.cat((Res_LR1.T.reshape(num_images*num_mines,1),Res_LR2.T.reshape(num_images*num_mines,1),Res_LR3.T.reshape(num_images*num_mines,1),Res_LR4.T.reshape(num_images*num_mines,1)),dim=1),dim=1) #.reshape((num_mines,4,num_images)),axis=1)

    # determines which mine image overlaps the base images
    result_all = torch.any(torch.cat((Res_LL,Res_UL,Res_UR,Res_LR),dim=0).reshape((4,num_mines*num_images)),dim=0).reshape((num_images,num_mines))
    
    # generates a list of mine images per base image
    type_t = torch.nonzero(result_all==True,as_tuple=True)
    type_t1 = type_t[0].to(device='cpu').numpy()+i
    type_t2 = type_t[1].to(device='cpu').numpy()
    type_n = np.append(type_t1.reshape(len(type_t1),1),type_t2.reshape(len(type_t2),1),axis = 1)
    # print(type_n)

    # If any corner overlaps, then flags base image as a mine image.
    result = torch.any(result_all,dim=1)
    print("Result: Non Mine Images : ",num_images - result.sum().item(),"/",num_images)

    IT_t[:,4] = result

    #saves results to file
    # result_df = pd.DataFrame(IT_t.to(device="cpu"))
    # result_df.to_csv("Non_Mine with Type.csv",mode="a",header=False,index=False)
    # type_df = pd.DataFrame(type_n)
    # type_df.to_csv("Types.csv",mode="a",header=False,index=False)                       


NameError: name 'torch' is not defined

In [ ]:
result_df['ID'] = np.arange(i,(i+num_samples))
display(result_df)

In [ ]:
IT_flagged = IT_sample.copy()
print(IT_flagged.shape)
IT_flagged[:,4] = result
print(IT_flagged.shape)
print(IT_flagged[0:100,:])


In [ ]:
print(result.sum().item())

In [10]:
hits = pd.read_csv("Types.csv")
print(hits)

          Image     ID1
0           868  199320
1           869  199320
2           996  205141
3           997  205141
4          1011  222292
...         ...     ...
590879  4936930  230453
590880  4936932  230454
590881  4936933  230454
590882  4936934  230455
590883  4936935  230455

[590884 rows x 2 columns]


In [ ]:
# creates a matrix of base images classified as mines with the types of images that overlap

MineLabels['ID1'] = MineLabels.index
display(MineLabels)
hits = pd.merge(left=hits,right=MineLabels,how='left',on ='ID1' )
display(hits)
# display(hits)
hits2 = hits[['Image','ID1','ID','Stage2']]
# display(hits2)
hits2 = hits2.rename(columns={'ID':"MineType"})
# display(hits2)
hits3 = hits2.pivot_table(values='ID1',aggfunc='count',columns=('MineType','Stage2'),index="Image")
hits3[np.isnan(hits3)] = 0
hits3 = hits3.astype(int)
hits3['Image'] = hits3.index
# display(hits3)
hits3.to_csv("Images with MineTypes.csv",index=False)    

,Image,ID1,ID,Stage2
0,868,199320,0,Active
1,869,199320,0,Active
2,996,205141,0,Active
3,997,205141,0,Active
4,1011,222292,1,Active
...,...,...,...,...
590879,4936930,230453,10,Active
590880,4936932,230454,10,Active
590881,4936933,230454,10,Active
590882,4936934,230455,10,Active


,Image,ID1,MineType,Stage2
0,868,199320,0,Active
1,869,199320,0,Active
2,996,205141,0,Active
3,997,205141,0,Active
4,1011,222292,1,Active
...,...,...,...,...
590879,4936930,230453,10,Active
590880,4936932,230454,10,Active
590881,4936933,230454,10,Active
590882,4936934,230455,10,Active


MineType         0                1                2                3         \
Stage2   Abandoned Active Abandoned Active Abandoned Active Abandoned Active   
Image                                                                          
868              0      1         0      0         0      0         0      0   
869              0      1         0      0         0      0         0      0   
996              0      1         0      0         0      0         0      0   
997              0      1         0      0         0      0         0      0   
1011             0      0         0      1         0      0         0      0   
...            ...    ...       ...    ...       ...    ...       ...    ...   
4936930          0      0         0      0         0      0         0      0   
4936932          0      0         0      0         0      0         0      0   
4936933          0      0         0      0         0      0         0      0   
4936934          0      0         0      0         0      0         0      0   
4936935          0      0         0      0         0      0         0      0   

MineType         4         ...     28     29     31     32     33     34  \
Stage2   Abandoned Active  ... Active Active Active Active Active Active   
Image                      ...                                             
868              0      0  ...      0      0      0      0      0      0   
869              0      0  ...      0      0      0      0      0      0   
996              0      0  ...      0      0      0      0      0      0   
997              0      0  ...      0      0      0      0      0      0   
1011             0      0  ...      0      0      0      0      0      0   
...            ...    ...  ...    ...    ...    ...    ...    ...    ...   
4936930          0      0  ...      0      0      0      0      0      0   
4936932          0      0  ...      0      0      0      0      0      0   
4936933          0      0  ...      0      0      0      0      0      0   
4936934          0      0  ...      0      0      0      0      0      0   
4936935          0      0  ...      0      0      0      0      0      0   

MineType     36     37     38    Image  
Stage2   Active Active Active           
Image                                   
868           0      0      0      868  
869           0      0      0      869  
996           0      0      0      996  
997           0      0      0      997  
1011          0      0      0     1011  
...         ...    ...    ...      ...  
4936930       0      0      0  4936930  
4936932       0      0      0  4936932  
4936933       0      0      0  4936933  
4936934       0      0      0  4936934  
4936935       0      0      0  4936935  

[48233 rows x 55 columns]